Coleta, pre processamento, divisao de dados, treinamento , hiperajuste, avaliacao

# Fine tuning - Treinamento

Após formatar os dados vamos começar o processo de fine tuning.



In [3]:
# habilitando a utilização dos arquivos do drive como fonte de dados
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/tech-challenge3'

Mounted at /content/drive


In [4]:

%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

Configurando o modelo

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit", # recuperando um modelo pré treinado llama 3
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # load_in_4_bit verdadeiro para que tenhamos melhor performance com a memória
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs       = data["input"]
    outputs      = data["output"]
    prompts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = prompt.format(instruction, input, output) + EOS_TOKEN
        prompts.append(text)
    return { "prompt" : prompts, }



from datasets import load_dataset

# recuperando o primeiro chunk de dados formatados
train_ds = load_dataset("json", data_files=f"{path}/trn_prepared.json", split='train')
train_ds = train_ds.map(formatting_prompts_func, batched = True)

print(train_ds)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 1390403
})


In [ ]:
select_ds =train_ds.select(range(3))
select_ds['prompt'][1]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a content for the title above.\n\n### Input:\nMog's Kittens\n\n### Output:\nJudith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.<|end_of_text|>"

Criamos a função de previsão abaixo para mostar o que o modelo esta prevendo dados as instruções para ele.

In [ ]:
from transformers import TextStreamer

def execute_previsao(model, tokenizer, instruction, input):

    prompt =f"""Below is an instruction for creating an Amazon book description, paired with a book title. Write a compelling description that highlights the key features and appeal of the book.
    \n### Instruction:\n Write a detailed and engaging book description for the title below. \n\n### Input:\n{input}\n\n### Output:\n"""

    # Preparar o tokenizer e os inputs para inferência
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Definir o TextStreamer
    text_streamer = TextStreamer(tokenizer)

    # Gerar a inferência usando o modelo
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)



Testando o modelo antes de ser treinado


In [ ]:
shuffled_ds = train_ds.shuffle(seed=42)  # O seed garante que o embaralhamento seja reproduzível
# Executando inferência para os três primeiros prompts
for i in range(5):
    execute_previsao(model, tokenizer, shuffled_ds['instruction'][i],shuffled_ds['input'][i] )

<|begin_of_text|>Below is an instruction for creating an Amazon book description, paired with a book title. Write a compelling description that highlights the key features and appeal of the book.
    
### Instruction:
 Write a detailed and engaging book description for the title below. 

### Input:
How to Babysit a Grandma

### Output:
How to Babysit a Grandma is a delightful children's book that explores the importance of family bonds and the joy of shared experiences. The book's charming illustrations and playful storytelling make it an ideal choice for young readers. This book will leave a lasting impression on children and adults alike, reminding us all of the special moments we share with loved ones.<|end_of_text|>
<|begin_of_text|>Below is an instruction for creating an Amazon book description, paired with a book title. Write a compelling description that highlights the key features and appeal of the book.
    
### Instruction:
 Write a detailed and engaging book description for 

Aparentemente ele se perde nas respostas

**Treinamento**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1390403 [00:00<?, ? examples/s]

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
6.352 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,390,403 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 4
\        /    Total batch size = 128 | Total steps = 10,862
 "-____-"     Number of trainable parameters = 41,943,040


OutOfMemoryError: CUDA out of memory. Tried to allocate 23.04 GiB. GPU 0 has a total capacity of 39.56 GiB of which 8.16 GiB is free. Process 43110 has 31.32 GiB memory in use. Of the allocated memory 30.75 GiB is allocated by PyTorch, and 67.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
#@title Salvando o modelo treinado no Drive
model.save_pretrained(f"{path}/lora_model")
tokenizer.save_pretrained(f"{path}/lora_model")

In [ ]:
#@title Carregando o modelo para usá-lo em novas previsões
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = f"{path}/lora_model",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

In [ ]:

# Executando inferência para os três primeiros prompts
for i in range(10):
    execute_previsao(model, tokenizer, shuffled_ds['instruction'][i],shuffled_ds['input'][i] )

In [ ]:
exit()